In [1]:
import pandas as pd
pd.set_option("display.max_column", None)

In [2]:
home_data = pd.read_csv("melb_data.csv")

In [3]:
home_data.shape

(13580, 21)

In [4]:
y = home_data.Price

In [5]:
X = home_data.drop("Price", axis = 1)

In [6]:
X.head()

,Suburb,Address,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [9]:
from sklearn.model_selection import train_test_split

X_train_full, X_val_full, y_train_full, y_val_full = train_test_split(X, y, 
                                                                      train_size = 0.8, 
                                                                      test_size = 0.2, 
                                                                      random_state = 1)

In [10]:
categorical_cols = [cname for cname in X_train_full.columns 
                   if X_train_full[cname] < 10 and 
                   X_train_full[cname].dtype == "object" ]

numerical_cols = [cname for cname in X_train_full.columns 
                  if X_train_full[cname].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numercial_cols
X_train = X_train_full[my_cols].copy()
X_val = X_val_full[my_cols].copy()

TypeError: '<' not supported between instances of 'str' and 'int'